<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

## Data Processing Workflow

In this workflow within PyCCAPT, we can crop the data, do the voltage and bowl calibration, calculate the 3d reconstruction, and do the ranging.
</div>

In [1]:
# Activate intractive functionality of matplotlib
%matplotlib ipympl
# Activate auto reload 
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
# import libraries
import os
import numpy as np
import pandas as pd
import subprocess
from ipywidgets import widgets
from IPython.display import display
from ipywidgets import fixed, interact_manual
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

# Local module and scripts
from pyccapt.calibration.calibration import widgets as wd
from pyccapt.calibration.data_tools import data_tools, data_loadcrop
from pyccapt.calibration.tutorials.tutorials_helpers import helper_calibration
from pyccapt.calibration.tutorials.tutorials_helpers import helper_data_loader
from pyccapt.calibration.tutorials.tutorials_helpers import helper_temporal_crop
from pyccapt.calibration.tutorials.tutorials_helpers import helper_special_crop
from pyccapt.calibration.tutorials.tutorials_helpers import helper_t_0_tune
from pyccapt.calibration.tutorials.tutorials_helpers import helper_mc_plot
from pyccapt.calibration.tutorials.tutorials_helpers import helper_3d_reconstruction
from pyccapt.calibration.tutorials.tutorials_helpers import helper_ion_selection
from pyccapt.calibration.tutorials.tutorials_helpers import helper_visualization
from pyccapt.calibration.tutorials.tutorials_helpers import helper_ion_list
from pyccapt.calibration.calibration import ion_selection
from pyccapt.calibration.calibration import share_variables
from pyccapt.calibration.leap_tools.leap_tools import read_rrng

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">
In case of recieving the error about pytable library, you have to install the pytables library with conda command. to do that you can open a new cell and copy the line below in it. Then just run it like other cells. The pytables library will be innstalled.
    
`!conda install --yes --prefix {sys.prefix} pytables`
</div>


<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">
In the cell below we create variable object. This object is used in many of the functions to share the data between functions in as easy way. 
</div>


In [2]:
variables = share_variables.Variables()

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">
    
By clicking on the button below, you can select the dataset file you want to work with. The dataset file can be in various formats, including HDF5, EPOS, POS, ATO, and CSV. The cropped data will be saved in the same directory as the original dataset file in a new directory nammed load_crop. The name of the cropped dataset file will be the same as the original dataset file. The figures will be saved in the same directory as the dataset file.
</div>

In [3]:
button = widgets.Button(description='load dataset')

@button.on_click
def open_file_on_click_r(b):
    global dataset_path
    folder_path = variables.last_directory
    script = '..//..//data_tools//run_dataset_path_qt.py'
    cmd = f"python {script} {folder_path}"
    result = subprocess.run(cmd, capture_output=True, text=True, shell=True)
    dataset_path = result.stdout.strip()
    variables.last_directory = dataset_path

button

Button(description='load dataset', style=ButtonStyle())

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

## ROI Selection and Data Cropping
    

</div>



<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

    
From the dropdown lists below, you can select the instrument specifications of the dataset. The instrument specifications are the same as the ones used for the calibration process. Data mode is specify the dataset structure. The dataset can be in raw or calibrated mode. The flight path length is the distance between the sample and the detector. The t0 is the time of flight of the ions with the lowest mass-to-charge ratio. The maximum mass-to-charge ratio is the maximum mass-to-charge ratio of tat you want to plot. You can also change it in te related cells. The detector diameter is the diameter of the detector.
</div>


In [4]:
# create an object for selection of instrument specifications of the dataset
tdc, pulse_mode, flight_path_length, t0, max_mc, det_diam = wd.dataset_instrument_specification_selection()

# Display lists and comboboxes to selected instrument specifications
display(tdc, pulse_mode, flight_path_length, t0, det_diam, max_mc)

Dropdown(description='Data mode:', options=('pyccapt', 'leap_epos', 'leap_pos', 'leap_apt', 'ato_v6'), value='…

Dropdown(description='Pulse mode:', options=('voltage', 'laser'), value='voltage')

FloatText(value=110.0, description='Flight path length:')

FloatText(value=38.0, description='t0:')

FloatText(value=80.0, description='Detector diameter:')

FloatText(value=400.0, description='Max mc:')

In [5]:
helper_data_loader.load_data(dataset_path, max_mc.value, flight_path_length.value, pulse_mode.value, tdc.value, variables)
data_tools.extract_data(variables.data, variables, flight_path_length.value, max_mc.value)
display(variables.data)
display(variables.range_data)

The maximum possible TOF is: 5010 ns
The data will be saved on the path: T:/Ott/APT_Measurements/Oxcart/2409_Feb-14-2025_15-08_NiC6_Pd139Charged2bar/2409_Feb-14-2025_15-08_NiC6_Pd139Charged2bar/data_processing/
The dataset name after saving is: 2409_Feb-14-2025_15-08_NiC6_Pd139Charged2bar
The figures will be saved on the path: T:/Ott/APT_Measurements/Oxcart/2409_Feb-14-2025_15-08_NiC6_Pd139Charged2bar/2409_Feb-14-2025_15-08_NiC6_Pd139Charged2bar/data_processing/
{'apt': ['experiment_chamber_vacuum', 'id', 'num_events', 'num_raw_signals', 'temperature', 'timestamps'], 'dld': ['high_voltage', 'laser_pulse', 'start_counter', 't', 'voltage_pulse', 'x', 'y'], 'tdc': ['channel', 'high_voltage', 'laser_pulse', 'start_counter', 'time_data', 'voltage_pulse']}
The data is loaded in raw mode
The number of data that is removed: 57285
Total number of Ions: 627777
The maximum possible time of flight is: 5010


,x (nm),y (nm),z (nm),mc (Da),mc_uc (Da),high_voltage (V),pulse,t (ns),t_c (ns),x_det (cm),y_det (cm),delta_p,multi,start_counter
0,0.0,0.0,0.0,0.0,0.0,500.000,328.0000,4661.128854,0.0,-2.429388,-0.646531,0.0,0.0,4270
1,0.0,0.0,0.0,0.0,0.0,500.000,328.0000,638.376930,0.0,0.235102,-3.353469,0.0,0.0,4466
2,0.0,0.0,0.0,0.0,0.0,500.000,328.0000,4558.793778,0.0,-1.766531,1.933061,0.0,0.0,7159
3,0.0,0.0,0.0,0.0,0.0,500.000,328.0000,4717.302732,0.0,3.141224,0.365714,0.0,0.0,109
4,0.0,0.0,0.0,0.0,0.0,500.000,328.0000,1018.824480,0.0,-1.564082,-3.340408,0.0,0.0,2883
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
627772,0.0,0.0,0.0,0.0,0.0,7178.760,1076.8140,3453.558498,0.0,-2.213878,-2.099592,0.0,0.0,10309
627773,0.0,0.0,0.0,0.0,0.0,7178.760,1076.8140,3892.477356,0.0,-0.143673,1.975510,0.0,0.0,11407
627774,0.0,0.0,0.0,0.0,0.0,7178.760,1076.8140,1073.709054,0.0,-3.585306,-1.071020,0.0,0.0,10644
627775,0.0,0.0,0.0,0.0,0.0,7183.688,1077.5532,1091.313540,0.0,-1.658776,1.498776,0.0,0.0,90


,name,ion,mass,mc,mc_low,mc_up,color,element,complex,isotope,charge
0,unranged0,un,0.0,0.0,0.0,400.0,#000000,[unranged],[0],[0],0


<div style="margin: 0 auto; padding: 20px;">

Load a pre saved range file if you want from cell below
</div>


In [6]:
button_r = widgets.Button(description='load range dataset')

@button_r.on_click
def open_file_on_click_r(b):
    global range_path
    folder_path = variables.last_directory
    script = '..//..//data_tools//run_dataset_path_qt.py'
    cmd = f"python {script} {folder_path}"
    result = subprocess.run(cmd, capture_output=True, text=True, shell=True)
    range_path = result.stdout.strip()
    variables.last_directory = range_path

button_r

Button(description='load range dataset', style=ButtonStyle())

In [7]:
if 'range_path' in globals():
    _, file_extension = os.path.splitext(range_path)
    if file_extension == '.rrng':
        variables.range_data = read_rrng(range_path)
    else:
        variables.range_data = pd.read_hdf(range_path, mode='r')
display(variables.range_data)

,name,ion,mass,mc,mc_low,mc_up,color,element,complex,isotope,charge
0,unranged0,un,0.0,0.0,0.0,400.0,#000000,[unranged],[0],[0],0


<div style="margin: 0 auto; padding: 20px;">

# Temporal crop

Select the data by drawing a rectangle over the experiment history. Experiment history is a 2D histogram of the time of flight of the ions versus sequence of evaporation. The experiment history is plotted by clicking on the button below te cell.
</div>


In [8]:
helper_temporal_crop.call_plot_crop_experiment(variables, pulse_mode.value)

Output()

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

# Spatial crop

Select the region of maximum concentration of Ions in the below plotted graph to utilize relevant data. To crop you can draw a circle over the filed desorption map. The field desorption map is a 2D histogram of the time of flight of the ions versus the position of the ions on the detector. The field desorption map is plotted by clicking on the button below the cell.
</div>

In [9]:
helper_special_crop.call_plot_crop_fdm(variables)

Output()

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

Calculate pulses since the last event pulse and ions per pulse. The percentage of loss in ROI selection process will be printed.
</div>

In [10]:
# maximum value of the tdc counter
if tdc.value == 'pyccapt':
    if not (variables.data['delta_p'] != 0).any() or not (variables.data['multi'] != 0).any():
        if (variables.data['start_counter'] != 0).any():
            max_start_counter = max(variables.data['start_counter'])
            # if (variables.data['pulse_pi'] == 0).all() or (variables.data['ion_pp'] == 0).all():
            pulse_pi, ion_pp = data_loadcrop.calculate_ppi_and_ipp(variables.data, max_start_counter)
            # add two calculated array to the croped dataset
            variables.data['delta_p'] = pulse_pi.astype(np.uintc)
            variables.data['multi'] = ion_pp.astype(np.uintc)
    
    # save the cropped data
    print('tof Crop Loss {:.2f} %'.format((100 - (len(variables.data) / len(variables.data_backup)) * 100)))
#percentage of double event per pulse
ion_pp = variables.data['multi'].to_numpy()
print('percentage of multihit event per pulse', len(ion_pp[ion_pp != 1]) / float(len(ion_pp)) * 100)

Progress: 0% complete
Progress: 19% complete
Progress: 39% complete
Progress: 59% complete
Progress: 79% complete
Progress: 99% complete
tof Crop Loss 28.47 %
percentage of multihit event per pulse 10.333186359000395


<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

In the next cell by changing the t<sub>0</sub> value you can correct the position of H<sub>1</sub> or any other known peak. this correction would be helpful for the position of the peaks in the m/c calibration process.
</div>

In [11]:
data_tools.extract_data(variables.data, variables, flight_path_length.value, max_mc.value)
variables.data

The maximum possible time of flight is: 5010


,x (nm),y (nm),z (nm),mc (Da),mc_uc (Da),high_voltage (V),pulse,t (ns),t_c (ns),x_det (cm),y_det (cm),delta_p,multi,start_counter
0,0.0,0.0,0.0,0.0,0.0,3263.544,489.5316,1122.414570,0.0,-2.494694,-0.355918,0,1,3240
1,0.0,0.0,0.0,0.0,0.0,3263.544,489.5316,1120.268016,0.0,-2.524082,-0.202449,8,3,3248
2,0.0,0.0,0.0,0.0,0.0,3263.544,489.5316,1576.832508,0.0,-2.452245,-0.039184,0,3,3248
3,0.0,0.0,0.0,0.0,0.0,3263.544,489.5316,1936.630620,0.0,-2.217143,-0.466939,0,3,3248
4,0.0,0.0,0.0,0.0,0.0,3263.544,489.5316,541.253934,0.0,2.448980,-1.064490,24,2,3272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449053,0.0,0.0,0.0,0.0,0.0,6035.768,905.3652,813.543966,0.0,1.440000,0.568163,5,1,2881
449054,0.0,0.0,0.0,0.0,0.0,6035.768,905.3652,1347.789024,0.0,-0.597551,0.963265,75,1,2956
449055,0.0,0.0,0.0,0.0,0.0,6035.768,905.3652,793.642050,0.0,-0.555102,-1.975510,53,1,3009
449056,0.0,0.0,0.0,0.0,0.0,6035.768,905.3652,1138.174254,0.0,2.697143,0.682449,96,1,3105


In [12]:
helper_t_0_tune.call_fine_tune_t_0(variables, flight_path_length, pulse_mode, t0)

Output()

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

Remove the data with m/c greater than max m/c and x, y, t = 0. Also add the needed colums for calibration. The data types of the final cropped dataset is displayed below.

</div>

In [13]:
# add the columns in the dataset for x, y, z, mc, mc calibrated, and t calibrated
helper_data_loader.add_columns(variables, max_mc)
# save data temporarily
data_tools.save_data(variables.data, variables, hdf=True, epos=False, pos=False, csv=False, temp=True)
display(variables.data)
display(variables.data.dtypes)

The number of data over max_mc: 0
The number of data with having t, x, and y equal to zero is: 0


,x (nm),y (nm),z (nm),mc (Da),mc_uc (Da),high_voltage (V),pulse,t (ns),t_c (ns),x_det (cm),y_det (cm),delta_p,multi,start_counter
0,0.0,0.0,0.0,0.0,64.675679,3263.544,489.5316,1122.414570,0.0,-2.494694,-0.355918,0,1,3240
1,0.0,0.0,0.0,0.0,64.387696,3263.544,489.5316,1120.268016,0.0,-2.524082,-0.202449,8,3,3248
2,0.0,0.0,0.0,0.0,130.865904,3263.544,489.5316,1576.832508,0.0,-2.452245,-0.039184,0,3,3248
3,0.0,0.0,0.0,0.0,200.806360,3263.544,489.5316,1936.630620,0.0,-2.217143,-0.466939,0,3,3248
4,0.0,0.0,0.0,0.0,13.726180,3263.544,489.5316,541.253934,0.0,2.448980,-1.064490,24,2,3272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449053,0.0,0.0,0.0,0.0,62.953747,6035.768,905.3652,813.543966,0.0,1.440000,0.568163,5,1,2881
449054,0.0,0.0,0.0,0.0,181.959936,6035.768,905.3652,1347.789024,0.0,-0.597551,0.963265,75,1,2956
449055,0.0,0.0,0.0,0.0,58.882083,6035.768,905.3652,793.642050,0.0,-0.555102,-1.975510,53,1,3009
449056,0.0,0.0,0.0,0.0,121.800024,6035.768,905.3652,1138.174254,0.0,2.697143,0.682449,96,1,3105


x (nm)              float64
y (nm)              float64
z (nm)              float64
mc (Da)             float64
mc_uc (Da)          float64
high_voltage (V)    float64
pulse               float64
t (ns)              float64
t_c (ns)            float64
x_det (cm)          float64
y_det (cm)          float64
delta_p              uint32
multi                uint32
start_counter        uint32
dtype: object

-------------------------------------------------------
-------------------------------------------------------

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

## Time-of-Flight/Mass-to-charge Calibration
</div>

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">
    
Below you can plot the ToF histogram of the dataset. You can select the peak range of the data you want to plot by drawing a rectangle over peak with holding left click. After that you should apply the voltage and bowl calibration. These three steps should be repeated until you see no improvement in the peak resolution. You can also save the calibration by clicking on the save button. The saved calibration will be used for the next steps.

</div>

In [14]:
# exctract needed data from Pandas data frame as an numpy array
data_tools.extract_data(variables.data, variables, flight_path_length.value, max_mc.value)

The maximum possible time of flight is: 5010


In [15]:
helper_calibration.call_voltage_bowl_calibration(variables, det_diam.value, flight_path_length.value, pulse_mode.value)

cannot find the maximum for i, j: 23 -20
cannot find the maximum for i, j: 13 -30
cannot find the maximum for i, j: 13 -30
cannot find the maximum for i, j: 23 -20
cannot find the maximum for i, j: 23 -20
cannot find the maximum for i, j: 13 -30
cannot find the maximum for i, j: 13 -30
cannot find the maximum for i, j: 23 -20
cannot find the maximum for i, j: 23 -20
cannot find the maximum for i, j: 23 -20
cannot find the maximum for i, j: 13 -30
cannot find the maximum for i, j: 23 -20
cannot find the maximum for i, j: -32 -20
cannot find the maximum for i, j: 13 -30


In [16]:
variables.dld_t_calib_backup = np.copy(variables.dld_t_calib)
variables.mc_calib_backup = np.copy(variables.mc_calib)

In [17]:
helper_ion_list.call_ion_list(variables, selector='peak')

In [19]:
variables.data['mc (Da)'] = variables.mc_calib
variables.data['t_c (ns)'] = variables.dld_t_calib
# Remove negative mc
threshold = 0
mc_t = variables.data['mc (Da)'].to_numpy()
mc_t_mask = (mc_t <= threshold)
print('The number of ions with negative mc are:', len(mc_t_mask[mc_t_mask==True]))
variables.data.drop(np.where(mc_t_mask)[0], inplace=True)
variables.data.reset_index(inplace=True, drop=True)
# save data temporarily
data_tools.save_data(variables.data, variables, hdf=True, epos=False, pos=False, csv=False, temp=True)

The number of ions with negative mc are: 0


In [20]:
helper_mc_plot.call_mc_plot(variables, selector='None')

Output()

-------------------------------------------------------
-------------------------------------------------------

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

## 3D Reconstruction
After bowl and voltage calibration we are ready to calculate the 3d reconstruction. In this workflow we calculate the reconstructed x,y,z and then plot the 3d, heatmap, projection plots and mass-to-charge histogram.

</div>

In [21]:
# exctract needed data from Pandas data frame as a numpy array
data_tools.extract_data(variables.data, variables, flight_path_length.value, max_mc.value)

display(variables.data)

The maximum possible time of flight is: 5010


,x (nm),y (nm),z (nm),mc (Da),mc_uc (Da),high_voltage (V),pulse,t (ns),t_c (ns),x_det (cm),y_det (cm),delta_p,multi,start_counter
0,0.0,0.0,0.0,55.261593,64.675679,3263.544,489.5316,1122.414570,1122.414570,-2.494694,-0.355918,0,1,3240
1,0.0,0.0,0.0,54.846865,64.387696,3263.544,489.5316,1120.268016,1120.268016,-2.524082,-0.202449,8,3,3248
2,0.0,0.0,0.0,110.505651,130.865904,3263.544,489.5316,1576.832508,1576.832508,-2.452245,-0.039184,0,3,3248
3,0.0,0.0,0.0,170.540124,200.806360,3263.544,489.5316,1936.630620,1936.630620,-2.217143,-0.466939,0,3,3248
4,0.0,0.0,0.0,12.158339,13.726180,3263.544,489.5316,541.253934,541.253934,2.448980,-1.064490,24,2,3272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449053,0.0,0.0,0.0,55.016520,62.953747,6035.768,905.3652,813.543966,813.543966,1.440000,0.568163,5,1,2881
449054,0.0,0.0,0.0,155.519648,181.959936,6035.768,905.3652,1347.789024,1347.789024,-0.597551,0.963265,75,1,2956
449055,0.0,0.0,0.0,53.299428,58.882083,6035.768,905.3652,793.642050,793.642050,-0.555102,-1.975510,53,1,3009
449056,0.0,0.0,0.0,106.296385,121.800024,6035.768,905.3652,1138.174254,1138.174254,2.697143,0.682449,96,1,3105


<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

You have to select the main element in your sample from the from dropdown below.
</div>

In [6]:
element_selected = wd.density_field_selection()
display(element_selected)

Dropdown(description='Element', options=((1, 'H', 52.7, 0.0), (2, 'He', 28.7, 0.0), (3, 'Li', 46.3, 14.0), (4,…

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

In case that yopu face error about plotly library, like javascripts error. You have check your jupyter lab version is compatibale with the plotly extenstion. Sometimes running `jupyter lab build` command fix the proble.
</div>

In [22]:
helper_3d_reconstruction.call_x_y_z_calculation(variables, flight_path_length, element_selected=element_selected)

Output()

In [23]:
variables.data['x (nm)'] = variables.x
variables.data['y (nm)'] = variables.y
variables.data['z (nm)'] = variables.z
# save data temporarily
data_tools.save_data(variables.data, variables, hdf=True, epos=False, pos=False, csv=False, temp=True)

-------------------------------------------------------
-------------------------------------------------------

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

## Ion Selection and Rangging

Users can choose ions using peak and element finders, manually add ions, customize ion colors, and create histograms. Histograms can be generated for selected ranges and areas, and figures can be saved. Additionally, users have the option to save both statistics and  range data.

</div>

In [24]:
# exctract needed data from Pandas data frame as an numpy array
data_tools.extract_data(variables.data, variables, flight_path_length.value, max_mc.value)

display(variables.data)

The maximum possible time of flight is: 12435


,x (nm),y (nm),z (nm),mc (Da),mc_uc (Da),high_voltage (V),pulse,t (ns),t_c (ns),x_det (cm),y_det (cm),delta_p,multi,start_counter
0,-0.737250,1.182940,0.051610,1.004139,0.786007,7767.467773,38548.199219,231.0663,233.957082,-0.756595,1.213980,0.0,0.0,0
1,0.392377,-3.502594,0.330121,89.755198,76.328761,7767.467773,38548.199219,1999.5467,1999.487180,0.406925,-3.632460,0.0,0.0,0
2,-2.973625,-0.000772,0.235211,130.713750,110.987143,7767.467773,38548.199219,2398.2223,2407.584075,-3.072740,-0.000798,0.0,0.0,0
3,-0.610765,1.647124,0.083283,567.301917,479.321347,7767.467773,38548.199219,4928.1093,4987.451239,-0.627502,1.692260,0.0,0.0,0
4,-0.299162,2.438246,0.161727,2.532462,2.044473,7767.467773,38548.199219,353.3698,357.056217,-0.308264,2.512430,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507188,0.779387,2.013662,265.263479,92.861373,78.118635,6539.762207,47708.601562,2192.0636,2033.345777,0.954383,2.465790,0.0,0.0,0
507189,-1.945079,-1.323470,265.291700,90.845656,77.369983,6539.762207,47708.601562,2183.5901,2011.439402,-2.384750,-1.622630,0.0,0.0,0
507190,1.239358,-2.259191,265.327313,92.927556,78.881532,6539.762207,47708.601562,2206.9331,2034.060981,1.521890,-2.774210,0.0,0.0,0
507191,0.553973,0.325295,265.132437,182.613097,154.149304,6539.762207,47708.601562,3053.0185,2840.951991,0.674327,0.395968,0.0,0.0,0


In [25]:
display(variables.data.dtypes)

x (nm)              float64
y (nm)              float64
z (nm)              float64
mc (Da)             float64
mc_uc (Da)          float64
high_voltage (V)    float64
pulse               float64
t (ns)              float64
t_c (ns)            float64
x_det (cm)          float64
y_det (cm)          float64
delta_p             float64
multi               float64
start_counter        uint32
dtype: object

In [26]:
helper_ion_selection.call_ion_selection(variables)

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

Now we can also show the color in the range dataframe

</div>

In [ ]:
display(variables.range_data.style.applymap(ion_selection.display_color, subset=['color']))

In [ ]:
variables.range_data.dtypes

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

Save the range in the hdf5 and csv format.

</div>

In [ ]:
interact_manual_range = interact_manual.options(manual_name="save range")
interact_manual_range(data_tools.save_range, variables=fixed(variables));

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

Save the cropped dataset. You can specify te output format from list below. The output formats are HDF5, EPOS, POS, ATO, and CSV. The output file will be saved in the same directory as the original dataset file in a new directory nammed load_crop.
</div>

In [22]:
interact_manual_data = interact_manual.options(manual_name="save data")
interact_manual_data(data_tools.save_data, data=fixed(variables.data), variables=fixed(variables),
                name=widgets.Text(value=variables.result_data_name),
                hdf=widgets.Dropdown(options=[('True', True), ('False', False)]),
                epos=widgets.Dropdown(options=[('False', False), ('True', True)]),
                pos=widgets.Dropdown(options=[('False', False), ('True', True)]),
                ato_6v=widgets.Dropdown(options=[('False', False), ('True', True)]),
                csv=widgets.Dropdown(options=[('False', False), ('True', True)]),
               temp=fixed(False));

interactive(children=(Text(value='2409_Feb-14-2025_15-08_NiC6_Pd139Charged2bar', description='name'), Dropdown…

-------------------------------------------------------
-------------------------------------------------------

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">
    
# Visualization
    
</div>

In [ ]:
variables.data

# helper_visualization.call_visualization(variables)

In [28]:
data_tools.extract_data(variables.data, variables, flight_path_length.value, max_mc.value)

The maximum possible time of flight is: 12435


In [29]:
helper_visualization.call_visualization(variables)

In [ ]:
variables.plotly_3d_reconstruction.show()

In [ ]:
from IPython.display import display, HTML
display(HTML(variables.animation_detector_html))